In [ ]:
# Bước 1: Đọc file + chia dữ liệu

In [11]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Đọc file CSV
df = pd.read_csv("f_coco_triplets.csv")

# Chia tập train/val/test
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

# Tạo entity & relation ID
entities = list(set(df['subject']).union(df['object']))
relations = list(set(df['predicate']))
entity2id = {e: i for i, e in enumerate(entities)}
relation2id = {r: i for i, r in enumerate(relations)}

# Chuyển về ID
def to_ids(dataframe):
    return [
        (entity2id[h], relation2id[r], entity2id[t])
        for h, r, t in zip(dataframe['subject'], dataframe['predicate'], dataframe['object'])
    ]

train_data = to_ids(train_df)
val_data = to_ids(val_df)
test_data = to_ids(test_df)

In [4]:
# Bước 2: Định nghĩa mô hình TransE

In [21]:
class TransEModel(nn.Module):
    def __init__(self, num_entities, num_relations, dim=200): 
        super(TransEModel, self).__init__()
        self.ent_embeddings = nn.Embedding(num_entities, dim)
        self.rel_embeddings = nn.Embedding(num_relations, dim)
        nn.init.xavier_uniform_(self.ent_embeddings.weight)
        nn.init.xavier_uniform_(self.rel_embeddings.weight)

    def forward(self, h, r, t):
        h_e = self.ent_embeddings(h)
        r_e = self.rel_embeddings(r)
        t_e = self.ent_embeddings(t)
        return torch.norm(h_e + r_e - t_e, p=1, dim=1)

In [22]:
# Bước 3: Huấn luyện mô hình

In [23]:
embedding_dim = 200
model = TransEModel(len(entities), len(relations), embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MarginRankingLoss(margin=0.5)  # margin mềm hơn

EPOCHS = 100
BATCH_SIZE = 512

# Convert toàn bộ train thành tensor để dễ batch
train_tensor = torch.tensor(train_data)

for epoch in range(EPOCHS):
    total_loss = 0
    perm = torch.randperm(len(train_tensor))

    for i in tqdm(range(0, len(train_tensor), BATCH_SIZE)):
        batch_idx = perm[i:i+BATCH_SIZE]
        batch = train_tensor[batch_idx]
        h_batch = batch[:, 0]
        r_batch = batch[:, 1]
        t_batch = batch[:, 2]

        # Negative sampling: chọn tail sai bất kỳ
        t_neg_batch = torch.randint(0, len(entities), t_batch.shape)

        pos_score = model(h_batch, r_batch, t_batch)
        neg_score = model(h_batch, r_batch, t_neg_batch)
        y = torch.tensor([-1], dtype=torch.float).repeat(pos_score.shape[0])
        loss = loss_fn(pos_score, neg_score, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}: Loss = {total_loss:.4f}")

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 166.80it/s]


Epoch 1/100: Loss = 2.8797


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 236.77it/s]


Epoch 2/100: Loss = 1.4733


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 403.04it/s]


...

Epoch 98/100: Loss = 0.0323


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 271.47it/s]


Epoch 99/100: Loss = 0.0589


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 252.74it/s]

Epoch 100/100: Loss = 0.0693


In [24]:
# Code đánh giá Precision@10, Recall@10, F1@10

In [25]:
import numpy as np

def evaluate_transe(model, test_data, entity2id, top_k=10):
    model.eval()
    hits = 0
    precision_total = 0
    recall_total = 0
    f1_total = 0

    with torch.no_grad():
        for h_id, r_id, true_t_id in tqdm(test_data[:500]):  # Giới hạn test 500 để chạy nhanh
            h = torch.tensor([h_id])
            r = torch.tensor([r_id])
            all_t = torch.arange(len(entity2id))

            # Lặp qua mọi tail candidate
            h_batch = h.repeat(len(all_t))
            r_batch = r.repeat(len(all_t))

            scores = model(h_batch, r_batch, all_t)
            topk_indices = torch.topk(scores, k=top_k, largest=False).indices  # chọn score thấp nhất

            if true_t_id in topk_indices:
                hits += 1
                precision_total += 1 / top_k
                recall_total += 1
                f1_total += 2 / (1 + top_k)  # F1@K = 2PR / (P+R)

    total = len(test_data[:500])
    precision = precision_total / total
    recall = recall_total / total
    f1 = f1_total / total

    print(f"Evaluation @Top-{top_k}:")
    print(f"Precision@{top_k}: {precision:.4f}")
    print(f"Recall@{top_k}: {recall:.4f}")
    print(f"F1@{top_k}: {f1:.4f}")

In [26]:
evaluate_transe(model, test_data, entity2id, top_k=10)

100%|██████████████████████████████████████████████████████████████████████████████| 332/332 [00:00<00:00, 1052.37it/s]

Evaluation @Top-10:
Precision@10: 0.0389
Recall@10: 0.3886
F1@10: 0.0706
